In [12]:
import pandas as pd
import matplotlib.pyplot as plt
import json
from collections import Counter
import ast
import networkx as nx
import re
from constants import base_path, model_list

In [13]:
with open(f"{base_path}\\input\\release_conditions.json") as f:
  disease_dict = json.load(f)
disease_list = list(disease_dict.keys())

In [14]:
def get_pred_miss_dict(error_df):
    pred_miss_dict = {}
    for disease in disease_list:
        miss_list = []
        for idx, row in error_df[error_df["PATHOLOGY"]==disease].iterrows():
            miss_list.extend(ast.literal_eval(row["predicted_diagnosis"]))
        miss_list = [i for i in miss_list if i!=disease]
        if miss_list:
            count = dict(Counter(miss_list))
            pred_miss_dict[disease]={i: round(count[i] / len(miss_list), 3) for i in count}
            # pred_miss_dict[disease]=dict(Counter(miss_list))
    return pred_miss_dict

In [15]:
def get_pred_miss_df(pred_miss_dict):
    pred_miss_df = pd.DataFrame({"disease": pred_miss_dict.keys()})
    for disease in disease_list:
        pred_miss_df[disease]=[pred_miss_dict[i].get(disease, 0) for i in pred_miss_dict.keys()]
    pred_miss_df.set_index('disease', inplace=True)
    return pred_miss_df

In [16]:
def get_pred_miss_graph(pred_miss_df):
    pred_miss_graph = pred_miss_df.stack()
    pred_miss_graph = pred_miss_graph.rename_axis(('Actual', 'Prediction Miss')).reset_index(name='weight')
    pred_miss_graph = pred_miss_graph[pred_miss_graph["weight"]>0]
    pred_miss_graph = pred_miss_graph.sort_values(['Actual','weight'], ascending=False)
    return pred_miss_graph

In [21]:
def analyze_errors(model_name):
    print(f"Analyzing {model_name}...")
    error_df = pd.read_csv(f"{base_path}\\output\\error_analysis_questionnaire\\{model_name}\\validation_df_all_patients_questionnaire.csv")
    error_df = error_df[error_df["is_matched"]==False]
    error_df = error_df[["PATHOLOGY", "predicted_diagnosis"]]
    error_df["PATHOLOGY"].value_counts().sort_values().plot.barh(figsize=(6, 8))
    plt.title(f"Prediction Error Frequency (w/ questionnaire)\n{model_name}")
    plt.xlabel("Count")
    save_path = f"{base_path}\\output\\error_analysis_questionnaire\\{model_name}"
    plt.savefig(f'{save_path}\\pred_miss_freq_questionnaire.jpg', bbox_inches='tight')
    plt.clf()
    pred_miss_dict = get_pred_miss_dict(error_df)
    pred_miss_df = get_pred_miss_df(pred_miss_dict)
    pred_miss_graph = get_pred_miss_graph(pred_miss_df)
    pred_miss_graph_sorted = pred_miss_graph.sort_values(by=['weight'], ascending=False)
    pred_miss_graph_sorted.to_csv(f"{save_path}\\pred_miss_weigths.csv", index=False)
    G = nx.from_pandas_edgelist(pred_miss_graph, 'Actual', 'Prediction Miss', edge_attr='weight', create_using=nx.DiGraph())
    pos = nx.spring_layout(G, seed=0)
    fig = plt.figure(figsize=(15, 8))
    weights = [G[u][v]['weight'] for u,v in G.edges()]
    nx.draw(G, pos, edge_color=weights, edge_cmap=plt.cm.Blues, with_labels=True, arrowsize=20)
    plt.title(f"Prediction Miss (w/ questionnaire)\n{model_name}", fontsize=20)
    plt.tight_layout()
    plt.savefig(f'{save_path}\\pred_miss_network.jpg', bbox_inches='tight')
    plt.clf()
    for disease in pred_miss_dict:
        G = nx.from_pandas_edgelist(pred_miss_graph[(pred_miss_graph["Actual"]==disease) | (pred_miss_graph["Prediction Miss"]==disease) ], 'Actual', 'Prediction Miss', edge_attr='weight', create_using=nx.DiGraph())
        pos = nx.spring_layout(G, seed=0)
        fig = plt.figure(figsize=(15, 8))
        weights = [G[u][v]['weight'] for u,v in G.edges()]
        nx.draw(G, pos, edge_color=weights, edge_cmap=plt.cm.Blues, with_labels=True, arrowsize=20)
        plt.title(f"Prediction Miss (w/ questionnaire) - {disease}\n{model_name}", fontsize=20)
        plt.tight_layout()
        img_filename = re.sub('[^a-zA-Z0-9 \n\.]', '', disease).replace(" ", "_")
        plt.savefig(f'{save_path}\\pred_miss_{img_filename}.jpg', bbox_inches='tight')
        plt.clf()

## Tree-based models

In [22]:
for model_name in model_list["tree-based"]:
    analyze_errors(model_name)
    # eval(model_name)

Analyzing decision_tree...


C:\Users\myras\AppData\Local\Temp\ipykernel_20216\4253896663.py:23: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\myras\AppData\Local\Temp\ipykernel_20216\4253896663.py:33: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\myras\AppData\Local\Temp\ipykernel_20216\4253896663.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(15, 8))


Analyzing random_forest...


C:\Users\myras\AppData\Local\Temp\ipykernel_20216\4253896663.py:23: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\myras\AppData\Local\Temp\ipykernel_20216\4253896663.py:33: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Analyzing gradient_boost...


...

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

## Logistic Regression

In [23]:
analyze_errors("logistic_regression")

Analyzing logistic_regression...


C:\Users\myras\AppData\Local\Temp\ipykernel_20216\4253896663.py:23: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\myras\AppData\Local\Temp\ipykernel_20216\4253896663.py:33: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
C:\Users\myras\AppData\Local\Temp\ipykernel_20216\4253896663.py:29: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = plt.figure(figsize=(15, 8))


<Figure size 600x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

...

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>

<Figure size 1500x800 with 0 Axes>